# 🧠 Chain-of-Thought for Clinical Reasoning: Practical Exercise

## Table of Contents
1. [Zero-Shot CoT: Simple Prompting](#practice-1-zero-shot-cot-simple-prompting)
2. [Few-Shot CoT: Learning from Examples](#practice-2-few-shot-cot-learning-from-examples)
3. [Self-Consistency: Multiple Reasoning Paths](#practice-3-self-consistency-multiple-reasoning-paths)
4. [Differential Diagnosis with CoT](#practice-4-differential-diagnosis-with-cot)
5. [Treatment Planning Chain](#practice-5-treatment-planning-chain)
6. [Drug Interaction Reasoning](#practice-6-drug-interaction-reasoning)
7. [Lab Result Interpretation](#practice-7-lab-result-interpretation)
8. [Clinical Note Generation](#practice-8-clinical-note-generation)

## Installing and Importing Essential Libraries

In [ ]:
# Import essential libraries
import anthropic
import os
from typing import List, Dict
import json
import time

# Visualization (optional for results)
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Settings
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
sns.set_style('whitegrid')

print("✅ All libraries loaded successfully!")
print("\n📌 Note: You'll need an Anthropic API key to run this notebook.")
print("   Set it as: os.environ['ANTHROPIC_API_KEY'] = 'your-key-here'")

## Setup: Initialize Claude API

We'll use Claude (Sonnet 4) for our Chain-of-Thought clinical reasoning exercises.

In [ ]:
# Initialize Anthropic client
# Make sure to set your API key first!
# os.environ['ANTHROPIC_API_KEY'] = 'your-api-key-here'

try:
    client = anthropic.Anthropic(api_key=os.environ.get("ANTHROPIC_API_KEY"))
    print("✅ Claude API initialized successfully!")
except Exception as e:
    print(f"❌ Error initializing API: {e}")
    print("Please set your ANTHROPIC_API_KEY environment variable.")

In [ ]:
# Helper function to call Claude
def call_claude(prompt: str, max_tokens: int = 2000, temperature: float = 0.7) -> str:
    """Call Claude API with given prompt"""
    try:
        message = client.messages.create(
            model="claude-sonnet-4-20250514",
            max_tokens=max_tokens,
            temperature=temperature,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return message.content[0].text
    except Exception as e:
        return f"Error: {str(e)}"

# Test the function
test_response = call_claude("Say 'Hello' if you're working!")
print(f"Test response: {test_response}")

---
## Practice 1: Zero-Shot CoT - Simple Prompting

### 🎯 Learning Objectives
- Understand the power of "Let's think step by step"
- Compare standard vs CoT prompting
- See how adding reasoning improves accuracy

### 📖 Key Concept
**Zero-Shot CoT:** Simply add "Let's think step by step" to elicit reasoning without examples

In [ ]:
# 1.1 Clinical case for testing
clinical_case = """
Patient presents with:
- Fever (39°C)
- Productive cough with yellow sputum
- Pleuritic chest pain (worse with breathing)
- Recent travel history

What is the most likely diagnosis?
"""

print("Clinical Case:")
print(clinical_case)

In [ ]:
# 1.2 Standard prompting (without CoT)
standard_prompt = clinical_case

print("=" * 60)
print("STANDARD PROMPTING (No CoT)")
print("=" * 60)

standard_response = call_claude(standard_prompt, temperature=0.3)
print(standard_response)
print("\n" + "=" * 60)

In [ ]:
# 1.3 Zero-Shot CoT prompting
cot_prompt = clinical_case + "\n\nLet's think step by step."

print("=" * 60)
print("ZERO-SHOT CoT PROMPTING")
print("=" * 60)

cot_response = call_claude(cot_prompt, temperature=0.3)
print(cot_response)
print("\n" + "=" * 60)

### 💡 Observation
Notice how adding "Let's think step by step" produces a more structured, systematic response with clear reasoning!

---
## Practice 2: Few-Shot CoT - Learning from Examples

### 🎯 Learning Objectives
- Provide reasoning examples to guide the model
- Structure clinical reasoning consistently
- Teach the format through demonstration

In [ ]:
# 2.1 Create Few-Shot examples
few_shot_examples = """
Here are examples of clinical reasoning:

Example 1:
Patient: 65-year-old male, sudden chest pain radiating to left arm, diaphoresis

Step 1: Identify key symptoms
- Chest pain with radiation
- Autonomic symptoms (sweating)
- Age >60 (risk factor)

Step 2: Generate differential diagnosis
- Myocardial infarction (most likely)
- Unstable angina
- Aortic dissection
- Pulmonary embolism

Step 3: Determine key discriminating tests
- ECG (looking for ST elevation)
- Troponin levels
- Cardiac enzymes

Step 4: Reach conclusion
Classic presentation of acute myocardial infarction. Immediate ECG and troponin needed.

---

Example 2:
Patient: 28-year-old female, severe headache, photophobia, neck stiffness, fever

Step 1: Identify key symptoms
- Severe headache
- Meningeal signs (neck stiffness, photophobia)
- Fever

Step 2: Generate differential diagnosis
- Bacterial meningitis (most concerning)
- Viral meningitis
- Subarachnoid hemorrhage
- Migraine with aura (less likely)

Step 3: Determine key discriminating tests
- Lumbar puncture with CSF analysis
- Blood cultures
- CT head (if focal signs)

Step 4: Reach conclusion
High suspicion for bacterial meningitis. Empiric antibiotics before LP if delay expected.

---

Now, analyze this new case using the same format:
"""

print("Few-Shot Examples Created!")

In [ ]:
# 2.2 New case to analyze
new_case = """
Patient: 45-year-old male, sudden severe abdominal pain (RLQ), nausea, vomiting, 
fever (38.5°C), rebound tenderness on examination.
"""

few_shot_prompt = few_shot_examples + new_case

print("=" * 60)
print("FEW-SHOT CoT RESPONSE")
print("=" * 60)

few_shot_response = call_claude(few_shot_prompt, max_tokens=2500, temperature=0.3)
print(few_shot_response)
print("\n" + "=" * 60)

---
## Practice 3: Self-Consistency - Multiple Reasoning Paths

### 🎯 Learning Objectives
- Generate multiple reasoning paths for the same case
- Compare and vote on different approaches
- Increase confidence through consensus

In [ ]:
# 3.1 Function to generate multiple reasoning paths
def generate_multiple_paths(case: str, n_paths: int = 5) -> List[str]:
    """Generate multiple reasoning paths with different temperature"""
    prompt = case + "\n\nLet's think step by step and provide a final diagnosis."
    
    paths = []
    for i in range(n_paths):
        print(f"Generating path {i+1}/{n_paths}...", end="\r")
        response = call_claude(prompt, temperature=0.7)  # Higher temp for diversity
        paths.append(response)
        time.sleep(1)  # Respect rate limits
    
    print(f"\n✅ Generated {n_paths} reasoning paths!")
    return paths

# Test case
test_case = """
Patient: 70-year-old female with diabetes, presenting with:
- Confusion (new onset)
- Fever (39.2°C)
- Hypotension (BP 85/50)
- Tachycardia (HR 125)
- Recent UTI history

What is the diagnosis?
"""

print("Test Case:")
print(test_case)
print("\nGenerating multiple reasoning paths...")

# Note: This will make 5 API calls, so it may take some time
reasoning_paths = generate_multiple_paths(test_case, n_paths=5)

In [ ]:
# 3.2 Display all paths
for i, path in enumerate(reasoning_paths, 1):
    print(f"\n{'='*60}")
    print(f"REASONING PATH {i}")
    print(f"{'='*60}")
    print(path)
    print()

In [ ]:
# 3.3 Extract and vote on diagnoses
def extract_diagnosis(text: str) -> str:
    """Simple extraction of final diagnosis (you can improve this)"""
    # Look for common diagnosis keywords
    text_lower = text.lower()
    if 'sepsis' in text_lower or 'septic' in text_lower:
        return 'Sepsis'
    elif 'urosepsis' in text_lower:
        return 'Urosepsis'
    elif 'infection' in text_lower:
        return 'Severe Infection'
    else:
        return 'Other'

diagnoses = [extract_diagnosis(path) for path in reasoning_paths]
diagnosis_counts = Counter(diagnoses)

print("\n" + "="*60)
print("SELF-CONSISTENCY VOTING RESULTS")
print("="*60)
for diagnosis, count in diagnosis_counts.most_common():
    percentage = (count / len(diagnoses)) * 100
    print(f"{diagnosis}: {count} votes ({percentage:.1f}%)")

winner = diagnosis_counts.most_common(1)[0][0]
print(f"\n🏆 Final Diagnosis (by consensus): {winner}")
print(f"   Confidence: {diagnosis_counts[winner]/len(diagnoses)*100:.1f}%")

---
## Practice 4: Differential Diagnosis with CoT

### 🎯 Learning Objectives
- Create systematic differential diagnoses
- Use probabilistic reasoning
- Update probabilities with new information

In [ ]:
# 4.1 Differential diagnosis template
differential_template = """
You are a clinical reasoning expert. Create a differential diagnosis with probabilities.

Patient Case:
{case}

Please provide:
1. Initial differential diagnosis (list 3-5 possibilities with estimated probabilities)
2. Key discriminating features for each diagnosis
3. Most likely diagnosis based on the information given
4. What additional test would most help narrow the diagnosis?

Format your response with clear sections and reasoning.
"""

# Example case: Chest pain
chest_pain_case = """
52-year-old male presents to ED with:
- Substernal chest pressure (7/10 severity)
- Started 2 hours ago while watching TV
- Associated with mild nausea
- Risk factors: Hypertension, smoking (20 pack-years), family history of CAD
- Vital signs: BP 145/90, HR 88, RR 18, O2 sat 98% on RA
"""

prompt = differential_template.format(case=chest_pain_case)

print("=" * 60)
print("DIFFERENTIAL DIAGNOSIS WITH PROBABILITIES")
print("=" * 60)

differential_response = call_claude(prompt, max_tokens=2500, temperature=0.3)
print(differential_response)

---
## Practice 5: Treatment Planning Chain

### 🎯 Learning Objectives
- Apply CoT to treatment decisions
- Consider drug selection, dosing, and monitoring
- Account for patient-specific factors

In [ ]:
# 5.1 Treatment planning prompt
treatment_case = """
Patient: 58-year-old male, newly diagnosed with hypertension
- BP readings: 155/95, 160/98, 152/94 (over 2 weeks)
- PMH: Type 2 diabetes (A1C 7.2%), no CAD
- Labs: Creatinine 1.1, eGFR 68, K+ 4.2
- Current medications: Metformin 1000mg BID
- Allergies: None known

Please develop a treatment plan with step-by-step reasoning:
1. List appropriate first-line medication options
2. Explain your drug choice based on patient factors
3. Specify initial dose and titration plan
4. Define monitoring parameters and follow-up timing
5. Identify any contraindications or precautions
"""

print("=" * 60)
print("TREATMENT PLANNING WITH CoT")
print("=" * 60)

treatment_response = call_claude(treatment_case, max_tokens=2500, temperature=0.3)
print(treatment_response)

---
## Practice 6: Drug Interaction Reasoning

### 🎯 Learning Objectives
- Identify potential drug interactions
- Reason about pharmacokinetic and pharmacodynamic interactions
- Propose safe alternatives

In [ ]:
# 6.1 Drug interaction case
drug_interaction_case = """
Patient on the following medications:
- Warfarin 5mg daily (INR target 2-3, current INR 2.5)
- Metoprolol 50mg BID
- Atorvastatin 40mg daily

New prescription being considered: Azithromycin 500mg daily x 5 days for pneumonia

Please analyze:
1. Identify potential drug interactions
2. Explain the mechanism of each interaction
3. Assess the clinical significance (minor/moderate/major)
4. Recommend management strategy:
   - Can azithromycin be safely used?
   - If yes, what monitoring is needed?
   - If no, what alternative antibiotic would you recommend?

Use step-by-step reasoning.
"""

print("=" * 60)
print("DRUG INTERACTION REASONING")
print("=" * 60)

interaction_response = call_claude(drug_interaction_case, max_tokens=2500, temperature=0.3)
print(interaction_response)

---
## Practice 7: Lab Result Interpretation

### 🎯 Learning Objectives
- Interpret complex lab patterns
- Connect lab findings to clinical presentations
- Generate appropriate follow-up recommendations

In [ ]:
# 7.1 Lab interpretation case
lab_case = """
42-year-old female presenting with fatigue for 3 months.

Lab Results:
- Hemoglobin: 8.5 g/dL (normal: 12-16)
- MCV: 68 fL (normal: 80-100)
- Ferritin: 8 ng/mL (normal: 15-150)
- TIBC: 450 μg/dL (normal: 250-400)
- Transferrin saturation: 8% (normal: 20-50%)

Please interpret these results step by step:
1. What type of anemia is this?
2. What is the underlying cause?
3. What are the most common etiologies to consider?
4. What additional workup is needed?
5. What treatment would you recommend?

Provide clear reasoning for each step.
"""

print("=" * 60)
print("LAB RESULT INTERPRETATION WITH CoT")
print("=" * 60)

lab_response = call_claude(lab_case, max_tokens=2500, temperature=0.3)
print(lab_response)

---
## Practice 8: Clinical Note Generation

### 🎯 Learning Objectives
- Generate structured clinical notes
- Include reasoning in the Assessment section
- Create comprehensive, professional documentation

In [ ]:
# 8.1 Clinical note generation
note_case = """
Generate a SOAP note for the following encounter:

Patient: 35-year-old male
Chief Complaint: "Severe sore throat for 3 days"

History:
- Sore throat started 3 days ago
- Difficulty swallowing, especially solids
- Fever (measured 38.9°C at home)
- No cough, no runny nose
- No sick contacts known
- PMH: None
- Medications: None
- Allergies: Penicillin (rash)

Physical Exam:
- Vital signs: T 38.7°C, BP 128/78, HR 88, RR 16
- General: Appears uncomfortable but not toxic
- HEENT: Pharyngeal erythema, tonsillar exudates bilaterally, tender anterior cervical lymphadenopathy
- Lungs: Clear bilaterally
- No rash

Please create a complete SOAP note with:
- Subjective section (organized)
- Objective section (organized by systems)
- Assessment section with differential diagnosis and clinical reasoning
- Plan section with specific treatments and follow-up

In the Assessment, include step-by-step reasoning about the diagnosis.
"""

print("=" * 60)
print("CLINICAL NOTE GENERATION WITH CoT REASONING")
print("=" * 60)

note_response = call_claude(note_case, max_tokens=3000, temperature=0.3)
print(note_response)

---
## 🎯 Practice Complete!

### Summary of What We Learned:

1. **Zero-Shot CoT**: Adding "Let's think step by step" dramatically improves reasoning
2. **Few-Shot CoT**: Providing examples teaches the model your preferred reasoning format
3. **Self-Consistency**: Multiple reasoning paths + voting increases confidence
4. **Clinical Applications**: CoT works for diagnosis, treatment, drug interactions, and more

### Key Performance Improvements:
- 📈 **15-80% accuracy increase** on complex medical reasoning tasks
- 🔍 **More transparent reasoning** - you can audit the logic
- ⚕️ **Safer clinical decisions** - systematic approach reduces errors
- 📝 **Better documentation** - reasoning is explicit and traceable

### Trade-offs:
- ⏱️ **2-3x slower** - more tokens generated
- 💰 **Higher API costs** - longer completions
- ✅ **But much more accurate and trustworthy!**

### Best Practices Recap:
1. ✅ Use clear, explicit step labels
2. ✅ Reference clinical guidelines when possible
3. ✅ Always verify reasoning logic
4. ✅ Include safety checks
5. ✅ Test on diverse cases
6. ❌ Don't use vague instructions
7. ❌ Don't skip validation steps
8. ❌ Never blindly trust outputs - human review is essential!

### Next Steps:
- Try these techniques on your own clinical cases
- Experiment with different prompt formulations
- Build custom few-shot example libraries for your specialty
- Integrate CoT into clinical decision support tools
- Always validate with domain experts!

### ⚠️ Critical Reminder:
**Chain-of-Thought prompting is a powerful tool to augment, not replace, clinical judgment. Always have outputs reviewed by qualified healthcare professionals.**

---
## 🔬 Bonus: Your Own Experiments

Use the cells below to try your own clinical cases!

In [ ]:
# Try your own case here!
my_case = """
[Enter your clinical case here]
"""

my_prompt = my_case + "\n\nLet's think step by step."

# Uncomment to run:
# response = call_claude(my_prompt, max_tokens=2500)
# print(response)

---
## 📚 References

- Wei et al. (2022). "Chain-of-Thought Prompting Elicits Reasoning in Large Language Models"
- Lecture 6: Chain-of-Thought for Clinical Reasoning
- Anthropic Claude API Documentation

---
**Created for:** Introduction to Biomedical Data Science  
**Instructor:** Ho-min Park  
**Contact:** homin.park@ghent.ac.kr | powersimmani@gmail.com